<a href="https://colab.research.google.com/github/abhinav-2003-bot/Cyberthon/blob/main/Training_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array

from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tsai.all import *
from tensorflow.keras.layers import LSTM
from IPython.display import clear_output
my_setup()

os              : Windows-10-10.0.19045-SP0
python          : 3.11.5
tsai            : 0.3.7
fastai          : 2.7.13
fastcore        : 1.5.29
torch           : 2.0.1+cpu
cpu cores       : 4
threads per cpu : 2
RAM             : 15.84 GB
GPU memory      : [4.0] GB


In [5]:
df=pd.read_csv('IMDB Dataset.csv')

In [6]:
df.shape
df.isnull().values.any()
df

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses main...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his ...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've ...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with so...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of conta...",positive
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, but who was expecting it to be. It was a whole lotta fun. the more i think about it the more i like it, and when it comes out on DVD I'm going to pay the money for it very proudly, every last cent. Sharon Stone is great, she always is, even if her movie is horrible(Catwoman), but this movie isn't, this is one of those movies that will be underrated for its lifetime, and it will probably become a classic in like 20 yrs. Don't wait for it to be a classic, watch it now and enjoy it. Don't expect a ...",positive
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ran continually over the overacted script, and a crappy copy of the VHS cannot be redeemed by consuming liquor. Trust me, because I stuck this turkey out to the end. It was so pathe

In [7]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [8]:
TAG_re=re.compile(r'<[^>]+>')

def remove_tags(text):
  '''Remove HTML tags: replace anything between opening  and closing <> with empty spaces'''
  return TAG_re.sub('',text)

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ABHINAV KUMAR
[nltk_data]     GARG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def preprocess_text(sen):
  sentence=sen.lower()
  #Remove html tags
  sentence = remove_tags(sentence)

  #Remve punctuation and numbers
  sentence = re.sub('[^a-zA-Z]', ' ',sentence)

  #Single character removal
  sentence = re.sub(r"\s+[a-zA-Z]\s+",  ' ',sentence)

  #Remove multiple spaces
  sentence = re.sub(r'\s+', ' ',sentence)

  #Remove multiple stopwards
  pattern = re.compile(r'\b('+ r'|'.join(stopwords.words('english'))+r')\b\s*')
  sentence = pattern.sub('', sentence)

  return sentence

In [11]:
X=[]
my_slice = slice(0, 1000, None)
selected_elements = df['review']
sentences = list(selected_elements)
for sen in sentences:
  X.append(preprocess_text(sen))

In [12]:
Y=df['sentiment']
Y=np.array(list(map(lambda x: 1 if x=="positive" else 0, Y)))

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, x_test, y_train,y_test= train_test_split(X,Y,test_size=0.2, random_state=42)

In [15]:
word_tokeniser = Tokenizer()
word_tokeniser.fit_on_texts(X_train)

X_train = word_tokeniser.texts_to_sequences(X_train)
X_test = word_tokeniser.texts_to_sequences(x_test)

In [16]:
vocab_length = len(word_tokeniser.word_index) + 1

vocab_length

92394

In [17]:
maxlen = 100
X_train = pad_sequences(X_train,padding='post',maxlen=maxlen)
x_test = pad_sequences(X_test,padding='post',maxlen=maxlen)


In [18]:
X, Y, splits = combine_split_data([X_train, x_test], [y_train, y_test])
X,Y,splits

(array([[[3.2471e+04, 1.2100e+02, 1.0000e+02, ..., 9.6000e+01,
          2.3200e+02, 3.6710e+03]],
 
        [[2.9900e+02, 3.3000e+01, 8.3000e+01, ..., 2.2000e+01,
          1.5450e+03, 2.7000e+01]],
 
        [[1.1730e+03, 3.5000e+01, 9.0000e+00, ...,        nan,
                 nan,        nan]],
 
        ...,
 
        [[3.0300e+02, 5.5400e+02, 2.4633e+04, ..., 4.8400e+02,
          7.4000e+01, 2.7920e+03]],
 
        [[1.1600e+02, 2.9000e+01, 5.1000e+01, ...,        nan,
                 nan,        nan]],
 
        [[1.0500e+02, 7.9180e+03, 5.8000e+01, ...,        nan,
                 nan,        nan]]]),
 array([0, 0, 1, ..., 1, 0, 1]),
 ((#40000) [0,1,2,3,4,5,6,7,8,9...],
  (#10000) [40000,40001,40002,40003,40004,40005,40006,40007,40008,40009...]))

In [19]:
X.shape,Y.shape,splits

((50000, 1, 100),
 (50000,),
 ((#40000) [0,1,2,3,4,5,6,7,8,9...],
  (#10000) [40000,40001,40002,40003,40004,40005,40006,40007,40008,40009...]))

### Model 1TSiTPlus)

In [20]:
tfms = [None, TSClassification()]
batch_tfms = TSStandardize()
fcst2 =  TSClassifier(X, Y,splits=splits, path='models', arch="TSiTPlus", tfms=tfms, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
fcst2.fit_one_cycle(5, 1e-3)
fcst2.export("TSiTPlus.pkl")

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 